In [1]:
import argparse
import json
import math
import os
from typing import Tuple

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchmetrics
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

In [ ]:
def seed_everything(seed=42):
    """設置隨機種子"""
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [2]:

    
def my_collate_fn(batch):
    data, labels = zip(*batch)
    
    max_length = max(len(item) for item in data)
    
    padded_data = [item + [0] * (max_length - len(item)) for item in data]
    
    padded_data = torch.tensor(padded_data, dtype=torch.float32).unsqueeze(2)
    labels = torch.tensor(labels, dtype=torch.float32).unsqueeze(1).unsqueeze(2)
    
    return padded_data, labels


In [3]:
class USG_Dataset(Dataset):
    def __init__(
        self,
        root: str,
        seed: int = 42,
        mode: str = "train",
        max_len: int = 400,
    ):
        self.root = root
        self.seed = seed
        self.mode = mode
        self.max_len = max_len  # 设定要统一的向量长度
        self.samples = [os.path.join(self.root, filename) for filename in os.listdir(self.root)]

    def to_tensor(self, x):
        return torch.tensor(x, dtype=torch.float32)
    
    # def pad_or_trim(self, array, target_len):
    #     if len(array) < target_len:
    #         padded_array = np.pad(array, (0, target_len - len(array)), 'constant')
    #     else:
    #         padded_array = array[:target_len]
    #     return padded_array

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path = self.samples[idx]
        with open(path, "r") as json_file:
            data = json.load(json_file)  
            x = data['sequence']
            label = data['label']


        # x_padded = self.pad_or_trim(x, self.max_len)

        # x_tensor = self.to_tensor(x).unsqueeze(0)
        # label_tensor = self.to_tensor(label).unsqueeze(0).unsqueeze(1)
        
        return x, label

In [4]:
class LSTMModel(nn.Module):
    def __init__(
        self,
        input_size,
        output_size,
        forecast_steps,
        hidden_size,
        num_layers,
        dropout=0.5,
    ):
        super(LSTMModel, self).__init__()
        self.output_size = output_size
        self.forecast_steps = forecast_steps

        self.embedding = nn.Linear(input_size, hidden_size)
        self.lstm_encoder = nn.LSTM(
            hidden_size,
            hidden_size,
            num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout,
        )
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size * 2, output_size * forecast_steps)

    def forward(self, x):
        x = self.embedding(x)
        out, (hidden, cell) = self.lstm_encoder(x, None)
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        out = out.view(-1, self.forecast_steps, self.output_size)
        return out

In [11]:
class Trainer:
    def __init__(
        self,
        model: nn.Module,
        lr: float,
        train_loader: DataLoader,
        val_loader: DataLoader,
    ):
        """初始化訓練器"""
        self.epochs_run = 0
        # 解析 opt 的必要參數
        self.lr = lr
        self.device = "cuda:1"
        self.eta_min = 1e-6
        self.max_epochs = 100
        # self.log_dir = opt.log_dir

        self.model = model.to(self.device)  # 模型
        self.train_loader = train_loader  # 訓練資料
        self.val_loader = val_loader  # 驗證資料
        # self.test_loader = test_loader  # 測試資料

        # self.criterion = nn.SmoothL1Loss()  # 損失函數
        self.criterion = nn.MSELoss()  # 損失函數
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        self.lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(
            self.optimizer, T_max=self.max_epochs, eta_min=self.eta_min, last_epoch=-1
        )
        # 指標計算器
        self.mse_metric = torchmetrics.MeanSquaredError().to(self.device)
        self.r2_metric = torchmetrics.R2Score().to(self.device)
        self.mae_metric = torchmetrics.MeanAbsoluteError().to(self.device)

        self.writer = SummaryWriter()

    def log_metrics(self, prefix: str, epoch: int):
        self.writer.add_scalar(f"{prefix}/MSE", self.mse_metric.compute().item(), epoch)
        self.writer.add_scalar(f"{prefix}/R2", self.r2_metric.compute().item(), epoch)
        self.writer.add_scalar(f"{prefix}/MAE", self.mae_metric.compute().item(), epoch)

        self.mse_metric.reset()
        self.r2_metric.reset()
        self.mae_metric.reset()

    def train_epoch(self, epoch: int):
        self.model.train()
        total_loss = 0.0
        pbar = tqdm(self.train_loader, desc=f"[{self.device}] Train Epoch {epoch:2d}")
        for src, tgt in pbar:
            src, tgt = src.to(self.device), tgt.to(self.device)
            self.optimizer.zero_grad()
            outputs = self.model(src)
            loss = self.criterion(outputs, tgt)
            loss.backward()
            self.optimizer.step()
            total_loss += loss.item()

            self.mse_metric(outputs, tgt)
            self.mae_metric(outputs, tgt)
            self.r2_metric(outputs.view(-1), tgt.view(-1))

        self.writer.add_scalar("Train/Loss", total_loss, epoch)
        self.writer.add_scalar(
            "Learning Rate", self.optimizer.param_groups[0]["lr"], epoch
        )
        self.log_metrics("Train", epoch)

    def val_epoch(self, epoch: int):
        self.model.eval()
        total_loss = 0.0
        with torch.no_grad():
            pbar = tqdm(self.val_loader, desc=f"[{self.device}] Val Epoch {epoch:2d}")
            for src, tgt in pbar:
                src, tgt = src.to(self.device), tgt.to(self.device)
                outputs = self.model(src)
                loss = self.criterion(outputs, tgt)
                total_loss += loss.item()

                self.mse_metric(outputs, tgt)
                self.mae_metric(outputs, tgt)
                self.r2_metric(outputs.view(-1), tgt.view(-1))

        self.writer.add_scalar("Val/Loss", total_loss, epoch)
        self.log_metrics("Val", epoch)

    # def test_epoch(self, epoch: int):
    #     self.model.eval()
    #     total_loss = 0.0
    #     with torch.no_grad():
    #         pbar = tqdm(self.test_loader, desc=f"[{self.device}] Test Epoch {epoch:2d}")
    #         for src, tgt in pbar:
    #             src, tgt = src.to(self.device), tgt.to(self.device)
    #             outputs = self.model(src)
    #             loss = self.criterion(outputs, tgt)
    #             total_loss += loss.item()

    #             self.mse_metric(outputs, tgt)
    #             self.mae_metric(outputs, tgt)
    #             self.r2_metric(outputs.view(-1), tgt.view(-1))

    #     self.writer.add_scalar("Test/Loss", total_loss, epoch)
    #     self.log_metrics("Test", epoch)

    def run(self):
        """主訓練循環"""
        for epoch in range(self.max_epochs):
            self.train_epoch(epoch)  # 訓練一個epoch
            self.lr_scheduler.step()  # 更新學習率調整器
            self.val_epoch(epoch)
            # self.test_epoch(epoch)


In [9]:
data_iter = iter(train_loader)
batch = next(data_iter)
inputs, labels = batch
print(inputs.shape)  # 应该是 (batch_size, max_len)

torch.Size([128, 429, 1])


In [12]:
seed = 87
lr = 1e-3
batch_size = 128
forecast_steps = 1

seed_everything(seed)
train_dataset = USG_Dataset(
    "./data/training",
    seed=seed,
    mode='train'
)
val_dataset = USG_Dataset(
    "./data/validation",
    seed=seed,
    mode='val'
)
# test_dataset = USG_Dataset(
#     "./data/raw_data.csv",
#     seed=seed,
#     mode='test'
# )
train_loader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, collate_fn=my_collate_fn
)
val_loader = DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, collate_fn=my_collate_fn
)
# test_loader = DataLoader(
#     test_dataset, batch_size=batch_size, shuffle=False, num_workers=4
# )
model = LSTMModel(
    input_size=1,
    output_size=1,
    forecast_steps=forecast_steps,
    hidden_size=128,
    num_layers=2,
)

trainer = Trainer(model, lr, train_loader, val_loader)#, test_loader)
trainer.run()

[cuda:1] Val Epoch 99: 100%|██████████| 3162/3162 [00:37<00:00, 84.99it/s]
